In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [2]:
path = "/home/kalyan/DataSets/animals/"

In [3]:
print(os.listdir(path))

['vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 'train.csv', 'trainLabels.npy', 'trainbeg.npy', 'test.csv', 'sample_submission.csv', 'testbeg.npy', 'self_vgg16.h5', 'test', 'train']


In [4]:
X_train = np.load(path + "trainbeg.npy")

In [5]:
X_test = np.load(path + "testbeg.npy")

In [6]:
Y_train =  np.load(path + "trainLabels.npy")
print(Y_train.shape)
Y_train = Y_train.reshape(Y_train.shape[0])
np.squeeze(Y_train)
print(Y_train.shape)
print(Y_train)

(13000, 1)
(13000,)
[11 26 10 ... 26 26 26]


Resnet Architecture

In [8]:
from keras.applications import ResNet50
from keras.models import Sequential, load_model, Model
from keras.layers import *
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

2023-04-08 11:47:56.196355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 11:47:57.042376: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.layers import Dense
from keras import regularizers

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

# Add a dense layer on top of the ResNet50 model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
predictions = Dense(30, activation='softmax')(x)

# Create the full model
model = Model(inputs=base_model.input, outputs=predictions)

In [18]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_4[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

In [19]:
model.compile(optimizer=Adam(lr=0.0001),
                loss='categorical_crossentropy',        
                metrics=['accuracy'])


/home/kalyan/miniconda3/lib/python3.10/site-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [20]:
model.fit(X_train/255., to_categorical(Y_train, num_classes=30), batch_size=64, epochs=10, verbose=1)

2023-04-08 11:50:05.667573: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.
2023-04-08 11:50:06.111693: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 638976000 exceeds 10% of free system memory.


Epoch 1/10


2023-04-08 11:50:11.224456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2023-04-08 11:50:13.184293: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 673.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-08 11:50:13.184344: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.34GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-04-08 11:50:13.503382: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 400.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains 

204/204 [==============================] - 36s 136ms/step - loss: 14.4476 - accuracy: 0.4005
Epoch 2/10
204/204 [==============================] - 27s 130ms/step - loss: 10.3265 - accuracy: 0.7527
Epoch 3/10
204/204 [==============================] - 27s 130ms/step - loss: 7.3759 - accuracy: 0.9195
Epoch 4/10
204/204 [==============================] - 27s 130ms/step - loss: 5.2589 - accuracy: 0.9655
Epoch 5/10
204/204 [==============================] - 27s 130ms/step - loss: 3.6922 - accuracy: 0.9768
Epoch 6/10
204/204 [==============================] - 27s 130ms/step - loss: 2.5419 - accuracy: 0.9807
Epoch 7/10
204/204 [==============================] - 27s 130ms/step - loss: 1.7838 - accuracy: 0.9665
Epoch 8/10
204/204 [==============================] - 27s 130ms/step - loss: 1.2482 - accuracy: 0.9609
Epoch 9/10
204/204 [==============================] - 27s 130ms/step - loss: 0.9171 - accuracy: 0.9548
Epoch 10/10
204/204 [==============================] - 27s 130ms/step - loss: 0.62

In [21]:
model.save(path + "self_resnet50.h5")

In [22]:
#predicting on test data
y_test_predict = model.predict(X_test/255.)
print(y_test_predict)


188/188 [==============================] - 6s 22ms/step
[[5.6558970e-06 4.6858077e-05 1.1881919e-02 ... 4.2412494e-04
  7.0931906e-06 1.4301834e-06]
 [4.6921517e-03 3.3467944e-04 3.2227337e-03 ... 2.0731888e-03
  1.3556154e-03 3.0760103e-04]
 [5.7905918e-01 1.1395101e-03 7.7154045e-03 ... 5.2771047e-03
  1.3814449e-01 1.6585598e-04]
 ...
 [2.9387014e-05 5.5083847e-01 9.4704889e-03 ... 8.3074541e-05
  6.5276829e-05 5.8108318e-04]
 [9.6382053e-07 1.7766629e-07 9.8776954e-06 ... 2.0847212e-08
  9.4356061e-08 6.3748342e-07]
 [2.5379664e-04 7.2680326e-05 7.8604020e-02 ... 1.0547791e-04
  1.4336433e-04 1.3985486e-04]]
